In [2]:
import pandas as pd

In [6]:
datas = pd.read_json('ngram/datas/bond_0517dirty.json')

In [7]:
datas.isnull().sum()

date    0
doc     0
dtype: int64

In [1]:
# Call Rate Crawling

import requests
from bs4 import BeautifulSoup
import pandas as pd

start_page = 1
end_page = 544

page_number = 1

date_list = []
rate_list = []

while True:
    URL = 'https://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CALL&page={}'.format(page_number)
    r = requests.get(URL)
    sp = BeautifulSoup(r.text,'html.parser')
    results = sp.select('tr')[1:]
    for result in results:
        date_list.append(result.select_one('td.date').text.strip())
        rate_list.append(result.select_one('td.num').text.strip())

    page_number += 1
    if page_number > end_page:
        break

call_df = pd.DataFrame(list(zip(date_list,rate_list)))

In [ ]:
call_df.columns = ['date','rate']

In [61]:
call_df

,rate
date,
2020-07-27,0.49
2020-07-24,0.48
2020-07-23,0.46
2020-07-22,0.45
2020-07-21,0.46
...,...
2005-03-30,3.25
2005-03-29,3.24
2005-03-28,3.25


In [7]:
import datetime

In [9]:
call_df['date'] = list(map(lambda i : datetime.datetime.strptime(i, '%Y.%m.%d'), list(call_df['date'])))

In [11]:
start_date = call_df['date'][len(call_df)-1]
end_date = call_df['date'][0]

print(start_date, end_date)


2005-03-24 00:00:00 2020-07-27 00:00:00


5604

In [64]:
call_df

,rate
date,
2020-07-27,0.49
2020-07-24,0.48
2020-07-23,0.46
2020-07-22,0.45
2020-07-21,0.46
...,...
2005-03-30,3.25
2005-03-29,3.24
2005-03-28,3.25


In [62]:
day_range = (end_date - start_date).days

temp = []
for i in range(day_range):
    temp.append((start_date + datetime.timedelta(i)).date())

call_final = pd.DataFrame(temp, columns = ['date'])

In [76]:
call_rate = call_final.set_index('date')
call_rate['rate'] = call_df.rate


In [80]:
call_final = call_rate.sort_values(by='date').fillna(method='pad')

In [84]:
call_final.to_json('ngram/target/call_raw.json')

In [184]:
# 금리 데이터 기간, rate limit 별로 저장

import pandas as pd
import datetime

datas = pd.read_json('ngram/target/call_raw.json')

date_count = 30
rate_limit = 0.03

temp = []
for i in range(len(datas)-date_count):
    rate_change = float(datas['rate'][i+date_count]) - float(datas['rate'][i])
    if rate_change >= rate_limit:
        temp.append('up')
    elif rate_change <= -rate_limit:
        temp.append('down')
    else:
        temp.append(0)

temp
new_data = datas.iloc[:-date_count]
new_data['ud'] = temp
new_data.reset_index(inplace=True)
new_data.columns = ['date', 'rate', 'ud']
new_data['date'] = list(map(lambda i : i.date(), new_data['date']))
new_data.to_json('ngram/target/call_{}_{}.json'.format(str(date_count), str(rate_limit)))

In [154]:
# Read
date_count = 30
rate_limit = 0.02
call_data = pd.read_json('ngram/target/call_{}_{}.json'.format(str(date_count), str(rate_limit)))

,date,rate,ud
40,2005-05-03,3.30,down
43,2005-05-06,3.27,down
44,2005-05-07,3.27,down
45,2005-05-08,3.27,down
47,2005-05-10,3.30,down
...,...,...,...
5559,2020-06-12,0.46,up
5560,2020-06-13,0.46,up
5563,2020-06-16,0.46,up
5567,2020-06-20,0.46,down


In [ ]:
call_date[(call_data['date'] >= datetime.date(2005,5,1))&(call_data['ud']!=0)]

In [182]:
ngram_df = pd.read_json('total_ngram.json')
ngram_df['date'] = list(map(lambda i : i.date(), ngram_df['date']))

In [187]:
rate_df = pd.read_json('ngram/target/call_30_0.03.json')
rate_df['date'] = list(map(lambda i : i.date(), rate_df['date']))

In [188]:
ngram_df.set_index('date')

In [190]:
ngram_df.set_index('date', inplace=True)

In [194]:
ngram_df['ud'] = rate_df.set_index('date')['ud']

In [195]:
ngram_df = ngram_df[['ngram','ud']]

In [207]:
rate_df[rate_df['date'] == datetime.date(2020,7,15)]

,date,rate,ud


In [212]:
range(len(ngram_df)-1, 0)

range(305072, 0)

In [213]:
ngram_df.reset_index(inplace=True)

In [214]:
ngram_df

,date,ngram,ud
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...",0
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...",down
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]",0
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...",up
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...",0
...,...,...,...
305068,2020-07-15,"[경기/NNG;침체/NNG, 완화/NNG]",NaN
305069,2020-07-15,"[코픽스/NNG;떨어/VV, 돈/NNG;확보/NNG, 하락/NNG, 내림/NNG]",NaN
305070,2020-07-15,"[저축성/NNG;예금/NNG;늘/VV, 경기/NNG;악화/NNG, 통화/NNG;급증...",NaN
305071,2020-07-15,"[최대/NNG, 충족/NNG]",NaN


In [221]:
for i in range(len(ngram_df)):
    if ngram_df['ngram'][i] == []:
        ngram_df.drop(i, axis=0, inplace=True)
ngram_df

,date,ngram,ud
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...",0
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...",down
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]",0
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...",up
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...",0
...,...,...,...
305067,2020-07-15,"[실망/NNG, 중단/NNG]",NaN
305068,2020-07-15,"[경기/NNG;침체/NNG, 완화/NNG]",NaN
305069,2020-07-15,"[코픽스/NNG;떨어/VV, 돈/NNG;확보/NNG, 하락/NNG, 내림/NNG]",NaN
305070,2020-07-15,"[저축성/NNG;예금/NNG;늘/VV, 경기/NNG;악화/NNG, 통화/NNG;급증...",NaN


In [223]:
ngram_df.dropna(inplace=True)

In [225]:
ngram_df.reset_index()[['date', 'ngram', 'ud']].to_json('train_data.json')

# Ngram 전처리

In [1]:
import pandas as pd
train_df = pd.read_json('train_data.json')
train_df['date'] = list(map(lambda i : i.date(), train_df['date']))

In [2]:
train_df['ngram']

0         [부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...
1         [경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...
2                          [대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]
3         [자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...
4         [부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...
                                ...                        
292539              [매출/NNG;감소/NNG, 피해/NNG, 최대/NNG, 개선/NNG]
292540    [경기/NNG;판단/NNG;상승/NNG, 경기/NNG;전망/NNG;하락/NNG, 전...
292541    [소비자/NNG;심리/NNG;긍정적/VAX, 지표/NNG;낮/VA, 하락/NNG, ...
292542    [주택가격/NNG;전망/NNG;상승/NNG, 취업기회/NNG;전망/NNG;오르/VV...
292543    [자산/NNG;가격/NNG;하락/NNG, 지수/NNG;저점/NNG;상승/NNG, 경...
Name: ngram, Length: 292544, dtype: object

In [3]:
train_df.isnull().sum()

date     0
ngram    0
ud       0
dtype: int64

In [4]:
import itertools
total_ngram = list(itertools.chain(*list(train_df['ngram'])))

In [5]:
len(total_ngram)

3197735

In [6]:
ngram_feature = set(total_ngram)

In [7]:
len(ngram_feature)

65447

In [8]:
counts = []
for nf in ngram_feature:
    counts.append(total_ngram.count(nf))

ngram_count = pd.DataFrame(list(zip(ngram_feature, counts)), columns = ['ngram', 'count'])

In [11]:
ngram_count.to_json('ngram_count.json')

In [10]:
ngram_count[ngram_count['count']<=15]

,ngram,count
1,우려/NNG;시장/NNG;경계/NNG,7
2,부동산/NNG;가격/NNG;잡/VV;금리/NNG;인상/NNG,3
4,헤알/NNG;환율/NNG;떨어/VV,10
7,기대/NNG;방어/NNG,4
10,gdp/NNG;성장률/NNG;시장/NNG;예상/NNG;하회/NNG,4
...,...,...
65434,소비/NNG;더디/VA;회복/NNG,7
65437,정책/NNG;확보/NNG,8
65440,지급불능/NNG;위험/NNG,12
65444,기업/NNG;많/VA;반대/NNG,1


In [17]:
import pandas as pd
import numpy as np
import itertools
import nltk

# 데이터프레임은 인덱스에 단어, 칼럼에 타겟

# input - split된 단어 list
class NBC():
    
    df = pd.DataFrame([])
    k = 0.5

    def __init__(self, **kwargs):
        try:
            self.k = kwargs['k']
        except:
            pass
    
    def add_data(self, datas):
        target_ls = list(set(datas['ud']))
        words = {}

        total_ngram = list(itertools.chain(*list(train_df['ngram'])))
        unique_ngram = list(set(total_ngram))

        result_df = pd.DataFrame(unique_ngram, columns = ['ngram']).set_index('ngram')

        for target in target_ls:
            this_ngram = list(itertools.chain(*list(datas[datas['ud'] == target]['ngram'])))
            fdist = nltk.FreqDist(this_ngram)
            temp_df = pd.DataFrame(list(zip(fdist.keys(), fdist.values())), columns= ['ngram', 'count']).set_index('ngram')
            result_df[target] = temp_df['count']

        self.df = result_df
        return self.df

    def pol_score(self, target, s):
        df = self.df
        k = self.k

        totals = {}
        for i in df:
            totals[i] = df[i].sum()
        
        tokens = s.split()
        total = np.asarray(list(totals.values())).sum()

        logs = {}
        for i in df:            
            logs[i] = np.log(float(totals[i]) / float(total))
        
        for token in tokens:
            for t in df:
                logs[t] += np.log( (k + df[t][token]) / (2*k +totals[t]) )

        bunmo = 0
        for log in logs:
            bunmo += np.exp(logs[log])
        bunja = np.exp(logs[target])
        return bunja / bunmo

In [18]:
nbc = NBC(k=0.05)

In [19]:
train_df

,date,ngram,ud
0,2005-05-12,"[부진/NNG;글로벌/NNG;경기/NNG;둔화/NNG;우려/NNG, 소비/NNG;설...",0
1,2005-06-09,"[경상/NNG;수지/NNG;적자/NNG;재정/NNG;적자/NNG, 국제/NNG;금융...",down
2,2005-06-23,"[대출/NNG;촉진/NNG, 걸림돌/NNG, 개선/NNG]",0
3,2005-07-07,"[자산/NNG;가격/NNG;상승/NNG;물가/NNG;상승/NNG, 물가/NNG;안정...",up
4,2005-08-11,"[부동산/NNG;가격/NNG;상승/NNG;금리/NNG;인상/NNG, 국채/NNG;수...",0
...,...,...,...
292539,2020-06-26,"[매출/NNG;감소/NNG, 피해/NNG, 최대/NNG, 개선/NNG]",0
292540,2020-06-26,"[경기/NNG;판단/NNG;상승/NNG, 경기/NNG;전망/NNG;하락/NNG, 전...",0
292541,2020-06-26,"[소비자/NNG;심리/NNG;긍정적/VAX, 지표/NNG;낮/VA, 하락/NNG, ...",0
292542,2020-06-26,"[주택가격/NNG;전망/NNG;상승/NNG, 취업기회/NNG;전망/NNG;오르/VV...",0


In [20]:
nbc.add_data(train_df)

,0,down,up
ngram,,,
경제/NNG;성장/NNG;낮/VA,21.0,15.0,2.0
신용/NNG;좋/VA,23.0,9.0,7.0
비농업/NNG;고용/NNG;시장/NNG;예상/NNG;하회/NNG,8.0,NaN,NaN
억제/NNG;넘/VV,NaN,1.0,NaN
강하/VA;추세/NNG,23.0,2.0,19.0
...,...,...,...
점차/MAG;금리/NNG;상승/NNG;압력/NNG;높/VA,1.0,NaN,NaN
자금/NNG;압박/NNG;심하/VA,7.0,8.0,1.0
물가/NNG;예상/NNG;벗어나/VV;않/VX,1.0,NaN,NaN


In [21]:
nbc.df.fillna(0)

,0,down,up
ngram,,,
경제/NNG;성장/NNG;낮/VA,21.0,15.0,2.0
신용/NNG;좋/VA,23.0,9.0,7.0
비농업/NNG;고용/NNG;시장/NNG;예상/NNG;하회/NNG,8.0,0.0,0.0
억제/NNG;넘/VV,0.0,1.0,0.0
강하/VA;추세/NNG,23.0,2.0,19.0
...,...,...,...
점차/MAG;금리/NNG;상승/NNG;압력/NNG;높/VA,1.0,0.0,0.0
자금/NNG;압박/NNG;심하/VA,7.0,8.0,1.0
물가/NNG;예상/NNG;벗어나/VV;않/VX,1.0,0.0,0.0


In [28]:
nbc.pol_score(0, '경제/NNG;성장/NNG;낮/VA')

0.5517693591307893

In [39]:
pd.read_json('test_data.json')

,date,sent
0,2020-06-24,관련부서는 금번 보고서 작성의 기본방향과 위원협의회에서 논의된 주요 내용 을 다음과...
1,2020-06-24,먼저 금번 보고서에서는 코로나19 확산에 따른 유례없는 금융·실물 충격이 금 융시스...
2,2020-06-24,다음으로 6월 11일 위원협의회에서 논의된 내용은 다음과 같음
3,2020-06-24,여러 위원들은 금번 보고서가 코로나19 확산 이후 금융불안 우려가 높아진 상황에서 ...
4,2020-06-24,일부 위원은 코로나19 확산 이후 금융시장 상황이 빠르게 변하고 있는 점을 감안하여...
...,...,...
38104,2005-05-12,"Dow Jones 지수는 미국의 1/4분기 GDP성장률 둔화(, 전분기 ) 등 경제..."
38105,2005-05-12,"유럽 주가는 유로지역 경제에 대한 비관적 전망, 인피니언사 등 주요 기업들의 실적부..."
38106,2005-05-12,유로화는 유로지역 경제 부진 전망 등으로 약세( 절하)를 보였으나 일본 엔화는 미국...
38107,2005-05-12,"미국의 장기금리는 미국의 경제성장률 둔화, 주가 하락 등으로 안전자산인 국채선호 경..."


In [38]:
train_df.columns[0]

'date'

In [7]:
nbc.df.to_json('train_result.json')

In [53]:
test_df['ngram'] = ngram_ls

In [55]:
# Ngram 빈 list 처리
train_df['ngram'] = list(map(lambda i : None if i == [] else i, train_df['ngram']))

test_df.isnull().sum()

date        0
sent        0
ngram    8687
dtype: int64

In [56]:
test_df.dropna()

,date,sent,ngram
1,2020-06-24,먼저 금번 보고서에서는 코로나19 확산에 따른 유례없는 금융·실물 충격이 금 융시스...,"[실물/NNG;충격/NNG, 금융안정/NNG]"
3,2020-06-24,여러 위원들은 금번 보고서가 코로나19 확산 이후 금융불안 우려가 높아진 상황에서 ...,"[불안/NNG;우려/NNG;높/VA, 높/VA;금융안정/NNG]"
6,2020-06-24,"일부 위원은 ‘금융안정 상황’ 및 ‘복원력’과 관련하여, 기업신용 상황을 분석하 는...",[금융안정/NNG]
7,2020-06-24,다른 일부 위원은 기업신용의 증가 원인을 수요와 공급 측면으로 나누어 기 술하는 것...,[신용/NNG;증가/NNG]
8,2020-06-24,이와 관련하여 일부 위원은 최근 기업대출 증가와 관련된 위험요인을 언급할 필요가 있...,[기업/NNG;대출/NNG;증가/NNG]
...,...,...,...
38104,2005-05-12,"Dow Jones 지수는 미국의 1/4분기 GDP성장률 둔화(, 전분기 ) 등 경제...","[둔화/NNG;경제/NNG;지표/NNG;부진/NNG, 지수/NNG;경제/NNG;지표..."
38105,2005-05-12,"유럽 주가는 유로지역 경제에 대한 비관적 전망, 인피니언사 등 주요 기업들의 실적부...","[기업/NNG;실적/NNG;부진/NNG;하락/NNG, 우려/NNG;기업/NNG;실적..."
38106,2005-05-12,유로화는 유로지역 경제 부진 전망 등으로 약세( 절하)를 보였으나 일본 엔화는 미국...,"[금리/NNG;인상/NNG;전망/NNG;상승/NNG, 유로존/NNG;경제/NNG;부..."
38107,2005-05-12,"미국의 장기금리는 미국의 경제성장률 둔화, 주가 하락 등으로 안전자산인 국채선호 경...","[금리/NNG;경제/NNG;지표/NNG;부진/NNG;하락/NNG, 안전자산/NNG;..."


In [60]:
temp = list(map(lambda i : ','.join(i), final_df['ngram']))

In [61]:
final_df['ngram'] = temp

In [64]:
final_df.reset_index()[['date','ngram']].to_json('test_ngram_datas.json')

In [82]:
import pandas as pd
import numpy as np
import itertools
import nltk
from sklearn.model_selection import train_test_split

# Input 데이터프레임은 첫 번째 column에 words(split처리 된 list), 두 번째 column에 target
# Output 데이터프레임은 인덱스에 단어, 칼럼에 타겟
class NBC():
    
    df = pd.DataFrame([])
    k = 0.5
    
    # 최초 모든 feature를 가지는 행렬 필요 initial_data
    def __init__(self, datas, **kwargs):
        self.raw_data = datas
        word_column = datas.columns[0]
        total_ngram = list(itertools.chain(*list(map(lambda i : i.split(','), datas[word_column])))) # list로 들어가지 않을 경우
        unique_ngram = list(set(total_ngram))
        self.init_df = pd.DataFrame(unique_ngram, columns = [word_column]).set_index(word_column)
        try:
            self.k = kwargs['k']
        except:
            pass
            
    # train data 받아서 count 행렬 반환하는 함수 #함수명 변경 
    def add_data(self, datas):
        word_column = datas.columns[0]
        target_column = datas.columns[1]
        target_ls = list(set(datas[target_column]))

        total_ngram = list(itertools.chain(*list(map(lambda i : i.split(','), datas[word_column])))) # list로 들어가지 않을 경우
        unique_ngram = list(set(total_ngram))

        result_df = pd.DataFrame(unique_ngram, columns = [word_column]).set_index(word_column)

        for target in target_ls:
            this_ngram = list(itertools.chain(*list(datas[datas[target_column] == target][word_column])))
            fdist = nltk.FreqDist(this_ngram)
            temp_df = pd.DataFrame(list(zip(fdist.keys(), fdist.values())), columns= [word_column, 'count']).set_index(word_column)
            result_df[target] = temp_df['count']

        result_df.fillna(0)
        self.df = result_df
        return self.df
    
    # real polarity Score 계산 함수 필요
    def polarity(self, df):
        c_df = self.add_data(df)
        
        pola_list = []
        # self.initial data set
        for i in c_df.index:
            try:
                pola = self.pol_score(c_df, 1, i) / self.pol_score(c_df, -1, i)
            except:
                pola = 1
            pola_list.append(pola)
        
        word_column = self.raw_data.columns[0]
        temp_df = pd.DataFrame(c_df.index, columns = [word_column])
        temp_df.set_index(word_column, inplace=True)
        temp_df['x'] = pola_list
        
        #initial data에 polarity score 추가
        self.init_df[len(self.init_df.columns)+1] = temp_df['x']

    # 30번 돌리는 것은 ngram feature의 polarity score, 평균내서 hawkish, dovish 구분
    # feature의 Polarity Score > 1 이면, hawkish else dovish 
    # polarity score 계산 시 1.3 이하면 neutral word로 (hawkish, neutral, dovish))

    # 최종 ngram 별 hawkish, dovish 사전 구축

    # Tone 계산 함수

    # for문 돌려서 feature 별 polarity score 계산
    # polarity score = pol_score('up') / pol_score('down')

    def pol_score(self, datas, target, s):
        df = datas
        k = self.k

        totals = {}
        for i in df:
            totals[i] = df[i].sum()
        
        tokens = s.split()
        total = np.asarray(list(totals.values())).sum()

        logs = {}
        for i in df:            
            if total == 0:
                return None
#                 logs[i] = 0
            else:
                logs[i] = np.log(float(totals[i]) / float(total))
        
        for token in tokens:
            for t in df:
                try:
                    logs[t] += np.log( (k + df[t][token]) / (2*k +totals[t]) )
                except:
                    logs[t] += np.log( (k) / (2*k +totals[t]) )

        bunmo = 0
        for log in logs:
            bunmo += np.exp(logs[log])
        bunja = np.exp(logs[target])
        return bunja / bunmo

    def fit_data(self):
        datas = self.raw_data
        for i in range(30):
            train_X, test_X, train_y, test_y = train_test_split(datas['ngram'], datas['ud'], 
                                                                shuffle=True, test_size=0.1)
            train_X = pd.DataFrame(train_X, columns = ['ngram'])
            train_X['ud'] = train_y
            self.polarity(train_X)
        result = pd.DataFrame(self.init_df.index, columns = ['ngram'])
        result['pol'] = self.init_df.mean(axis = 1)

        return result


    def predict(self, test_datas):

        return

In [57]:
ngram_df = pd.read_json('final_ngram_comma.json')
ngram_df['date'] = list(map(lambda i : i.date(), ngram_df['date']))
ngram_df.set_index('date', inplace=True)

In [58]:
cd_df = pd.read_json('labeled_cd_rate.json')
cd_df['date'] = list(map(lambda i : i.date(), cd_df['date']))
cd_df.set_index('date', inplace=True)

In [59]:
ngram_df['ud'] = cd_df['ud']

In [83]:
import datetime
nbc = NBC(ngram_df.iloc[(ngram_df.index >= datetime.date(2005,5,1)) & (ngram_df.index <= datetime.date(2017,12,31))])

In [84]:
results = nbc.fit_data()

In [88]:
results

,ngram,pol
0,민간/NNG;고용/NNG;부진/NNG,NaN
1,경제주체/NNG;심리/NNG;부정적/VAX,NaN
2,금리/NNG;서둘/VV;오르/VV,NaN
3,채권시장/NNG;호조/NNG,NaN
4,투자/NNG;위험/NNG;크/VV,NaN
...,...,...
64724,부동산/NNG;투자/NNG;급증/NNG,NaN
64725,하락/NNG;채권/NNG;금리/NNG;급등/NNG,NaN
64726,초호황/NNG,NaN
64727,수요/NNG;둔화/NNG;우려/NNG,NaN


In [89]:
np.array([np.nan, 1,2,3,4,5]).mean()

nan

In [91]:
nbc.init_df.head(50)

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
ngram,,,,,,,,,,,,,,,,,,,,,
민간/NNG;고용/NNG;부진/NNG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
경제주체/NNG;심리/NNG;부정적/VAX,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
금리/NNG;서둘/VV;오르/VV,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
채권시장/NNG;호조/NNG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
투자/NNG;위험/NNG;크/VV,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
채권/NNG;가격/NNG;하락/NNG;크/VV,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
유로존/NNG;위험/NNG;증대/NNG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
지표/NNG;발표/NNG;경기/NNG;회복/NNG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
irs/NNG;금리/NNG;급락/NNG,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
